## youtube8Mのデータセット準備

とりあえずローカルでDatasetAPIで学習を開始できるようにするのが目的

### ローカルへのデータダウンロード

youtube8mのデータは以下のコマンドで取って来れます。
下のセルを実行すると全データの1/100のトレーニングデータを取得します。

環境変数からオプションを渡すような作りになってるみたい。
このpartition=2の部分がよくわからないけど、とりあえずこのままでw

In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
BUCKET="asl-mixi-project-bucket"

In [3]:
%%bash 
CURRENT_DIR=$(PWD)
mkdir ./data/video/
cd ./data/video/
curl data.yt8m.org/download.py > download.py

shard=1,100 partition=2/video/train mirror=us python download.py
cd $CURRENT_DIR

Resuming Download ...
Files remaining 0
All done. No more files to download.


bash: line 1: PWD: command not found
mkdir: cannot create directory ‘./data/video/’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0   7889      0 --:--:-- --:--:-- --:--:--  7890


In [4]:
!ls ./data/video/*.tfrecord -l | wc -l

41


In [5]:
!ls ./data/video/*.tfrecord -alh | head

-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0093.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.7M Dec  6 08:34 ./data/video/train0111.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0208.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.7M Dec  6 08:34 ./data/video/train0274.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.4M Dec  6 08:34 ./data/video/train0276.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0352.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0434.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0477.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0503.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0580.tfrecord


### storageへのコピー

必要があれば実行。ローカルで実行する分には必要ありません。

In [ ]:
import os
os.environ['BUCKET'] = BUCKET

!gsutil cp ./data/video/*.tfrecord gs://$BUCKET/data/youtube-8m/

### データセットの作成用のコード

In [6]:
import tensorflow as tf

print(tf.__version__)

1.15.0


In [7]:
tf.enable_eager_execution()

In [8]:
from data import read_dataset

In [9]:
ds = read_dataset("./data/video/*.tfrecord", tf.estimator.ModeKeys.TRAIN, batch_size=2)

In [10]:
for row in ds.take(1):
    print(row)

({'mean_rgb': <tf.Tensor: id=68, shape=(2, 1024), dtype=float32, numpy=
array([[ 1.373725  ,  0.68594223, -0.5504293 , ..., -0.62593025,
        -0.29531184,  0.19984008],
       [ 0.24126838, -0.22392769, -0.40628064, ...,  0.03575367,
        -0.09794731, -0.25015318]], dtype=float32)>, 'id': <tf.Tensor: id=66, shape=(2,), dtype=string, numpy=array([b'BJ8D', b'168D'], dtype=object)>, 'mean_audio': <tf.Tensor: id=67, shape=(2, 128), dtype=float32, numpy=
array([[-8.70360315e-01, -7.25822508e-01,  1.46465373e+00,
        -2.57819921e-01, -1.03489387e+00,  1.77717257e+00,
        -1.28906322e+00,  7.10747316e-02, -5.47767209e-04,
        -1.91772079e+00, -9.11558330e-01, -5.77750981e-01,
         7.72561491e-01,  1.28476143e-01,  5.48299491e-01,
        -5.88093579e-01,  3.70923907e-01,  2.55776286e-01,
         1.83466017e+00, -1.59220472e-01, -3.02637845e-01,
         9.67519462e-01,  2.44227067e-01, -1.87686753e+00,
         1.17506099e+00, -2.14036241e-01, -4.54932600e-01,
         